In [23]:
import numpy as np
import sys
import copy
import PySimpleGUI as sg

In [32]:
draw_interface()

Citesc datele...
[111.75 105.75 110.   ... 107.75 112.75 106.5 ]
[ 1.25  2.75  1.75 ...  3.75  2.25 13.25]
Am terminat citirea, incep sa rezolv
ITERATION  1
ITERATION  2
ITERATION  3
ITERATION  4
ITERATION  5
ITERATION  6
ITERATION  7
ITERATION  8
ITERATION  9
ITERATION  10
Solutia a convers
Error = 6.774533318321119e-06
Primele 5 elemente din solutie:
[1. 1. 1. 1. 1.]
Citesc datele...
[111.75 105.75 110.   ... 108.25 105.5  109.25]
[ 9.75  5.5   8.25 ... 12.5   5.25  1.  ]
Am terminat citirea, incep sa rezolv
ITERATION  1
ITERATION  2
ITERATION  3
ITERATION  4
ITERATION  5
ITERATION  6
ITERATION  7
ITERATION  8
ITERATION  9
ITERATION  10
Solutia a convers
Error = 1.514638427124737e-06
Primele 5 elemente din solutie:
[0.33333333 0.33333333 0.33333333 0.33333333 0.33333333]
Citesc datele...
[16.25 10.25 14.5  ... 14.25 11.75 17.  ]
[ 4.25  4.25  2.75 ... 11.75  5.    7.  ]
Am terminat citirea, incep sa rezolv
ITERATION  1
ITERATION  2
ITERATION  3
ITERATION  4
ITERATION  5
ITERATION  6


In [24]:
def draw_interface():
    global l, window, optimized_flag, epsilon
 

    layout = [[sg.Text("Introduceti epsilon:")], 
              [sg.In(size=(25, 1), enable_events=True, key="EPS")],
              [sg.Button("GATA")],
        
              [sg.Button("TEST CASE 1")],
              [sg.Text(size=(100, 1), key="1")],
              
              [sg.Button("TEST CASE 2")],
              [sg.Text(size=(100, 1), key="2")],

              [sg.Button("TEST CASE 3")],
              [sg.Text(size=(100, 1), key="3")],

              [sg.Button("TEST CASE 4")],
              [sg.Text(size=(100, 1), key="4")],
              
              [sg.Button("TEST CASE 5")],
              [sg.Text(size=(100, 1), key="5")],
              
              [sg.Button("TEST CASE 6")],
              [sg.Text(size=(100, 1), key="6")],

              [sg.Button("EXIT")]
             ]
    window = sg.Window("Calcul numeric - Laborator 4", layout)

    while True:
        event, values = window.read()

        if event == "EXIT":
            break
        elif event == "TEST CASE 1":
            window["1"].update(solve(1))
        elif event == "TEST CASE 2":
            window["2"].update(solve(2))
        elif event == "TEST CASE 3":
            window["3"].update(solve(3))
        elif event == "TEST CASE 4":
            window["4"].update(solve(4))
        elif event == "TEST CASE 5":
            window["5"].update(solve(5))
        elif event == "TEST CASE 6":
            window["6"].update(solve(6))
        elif event == "GATA":
            epsilon = float(values["EPS"])

    window.close()

In [25]:
epsilon = 10 ** (-8)

def solve(test_case):
    global abc
    
    print('Citesc datele...')
    (a,b,c,f,n,p,q) = read_data("./a" + str(test_case) + ".txt", "./f" + str(test_case) + ".txt")
    print('Am terminat citirea, incep sa rezolv')
    if validate_data(a) == -1:
        print('Eroare: determinant = 0')
        sys.exit()
    abc = a
    abc = np.append(abc, b)
    abc = np.append(abc, c)

    solution = getSolutionGaussSeidel(n, f, p, q, 100)
    
    print('Error = ' + str(get_average_error(abc, n, p, q, f, solution)))
    
    print('Primele 5 elemente din solutie:')
    
    if n > 5:
        print(solution[0:5])
    else:
        print(solution[0:n])
    
    if n > 5:
        return solution[0:5]
    else:
        return solution[0:n]
    

In [26]:
def get_average_error(abc, n, p, q, f, solution):
    error = 0
    maxim = -1
    for i in range(0, n):
        value = 0
        if p + i <= n-1:
            value += matrix_to_array(n,p,q,i,p+i) * solution[p+i]
        if i >= q:
            value += matrix_to_array(n,p,q,i,i-q) * solution[i-q]
        value += matrix_to_array(n,p,q,i,i) * solution[i]
        error += abs(f[i] - value)
        if error > maxim:
            maxim = error
    return maxim

In [27]:
def getSolutionGaussSeidel(n, f, p, q, number_iterations):
    global epsilon, abc
    
    x_current = np.zeros(n)
                
    for j in range(1, number_iterations):
        print('ITERATION ', j)
        
        diff = 0
        
        for i in range(0, n):
            suma1 = 0
            suma2 = 0
#             for t in range(0, i):
            if p + i <= n-1:
                suma1 = matrix_to_array(n,p,q,i,p+i) * x_current[p+i]
#                 suma1 += (matrix_to_array(n, p, q, i, t) * x_current[t])
#             for t in range(i+1, n):
#                 suma2 += (matrix_to_array(n, p, q, i, t) * x_prev[t])
            if i >= q:
                suma2 = matrix_to_array(n,p,q,i,i-q) * x_current[i-q]

            final_value = (f[i] - suma1 - suma2) / matrix_to_array(n, p, q, i, i)
            diff += abs(final_value - x_current[i])
            x_current[i] = final_value
            
        if diff < epsilon:
            print('Solutia a convers')
            return x_current
        
    print('Divergenta!')
    return x_current

In [28]:
def read_data(file_name1, file_name2):
    file_read = open(file_name1, "r")
    a = np.array([])
    b = np.array([])
    c = np.array([])
    f = np.array([])
    n = int(file_read.readline())
    p = int(file_read.readline())
    q = int(file_read.readline())
    file_read.readline()
    for i in range(0, n):
        a = np.append(a, float(file_read.readline()[0:-1]))
    print(a)
    file_read.readline()
    for i in range(0, n-q):
        c = np.append(c, float(file_read.readline()[0:-1]))
    file_read.readline()
    for i in range(0, n-p):
        b = np.append(b, float(file_read.readline()[0:-1]))
    print(b)
    


        
    file_read2 = open(file_name2, "r")
    file_read2.readline()
    file_read2.readline()
    for i in range(0, n):
        f = np.append(f, float(file_read2.readline()[0:-1]))
    
    return (a,b,c,f,n,p,q)

In [29]:
def validate_data(a):
    for i in range(0, len(a)):
        if a[i] == 0:
            return -1
    return 1

In [30]:
def matrix_to_array(n, p, q, i, j):
    global abc
    if i != j and i+p != j and j+q != i:
        return 0
    if i == j:
        return abc[i]
    elif i < j:
        return abc[n + i]
    else:
        return abc[n + (n-p) + j]